# Install Libraries

In [1]:
!pip install selenium

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import pandas as pd

# Work

In [3]:
binary = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options = Options()
options.headless = True
options.binary = binary

cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True #optional

driver = webdriver.Firefox(options=options, capabilities=cap)
driver.get("http://diariooficial.rn.gov.br/dei/dorn3/")

In [8]:
materias = []

def navegar(link):
    link.click()
    global materias
    subsessoes = []
    try:
        subsessoes = driver.find_element_by_id("lblQtdMateriasPublicadas")
        if subsessoes.text == 'Nenhuma matéria encontrada nessa data para essa seção.':
            caminho = driver.find_element_by_id("lblSessaoPath").text
            lista_caminho = caminho.split("\\")
            lista_caminho.append(subsessoes.text)
            materias.append(lista_caminho)
            return []
        tabela = driver.find_element_by_id("dgDocumentos")
        files = tabela.find_elements_by_tag_name("a")
        for f in files:
            link = f.get_attribute("href")
            caminho = driver.find_element_by_id("lblSessaoPath").text
            lista_caminho = caminho.split("\\")
            lista_caminho.append(link)
            materias.append(lista_caminho)
        return []
    except:
        tabela = driver.find_element_by_id("dgSubSessoes")
        links = tabela.find_elements_by_tag_name("a")
    return links

#elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
#teste = navegar(elements)
#materias

[['DIÁRIO OFICIAL',
  'MINISTÉRIO PÚBLICO',
  '',
  'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190924&id_doc=659987'],
 ['DIÁRIO OFICIAL',
  'MINISTÉRIO PÚBLICO',
  '',
  'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190924&id_doc=659988']]

In [101]:
elements = driver.find_element_by_id('Menu1_dgMenu')
sessoes = elements.find_elements_by_tag_name('a')
lista = []
for s in sessoes:
    lista.append(navegar(s))
    print(lista)

[[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="a83d242f-f71d-47fd-8abe-5e70528df294", element="8c605c17-6608-49d5-b790-cdec1e82bdc3")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="a83d242f-f71d-47fd-8abe-5e70528df294", element="bc769307-0d04-4ce0-a1c8-c85cf0a6cdd7")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="a83d242f-f71d-47fd-8abe-5e70528df294", element="fffd5e52-4b03-4506-bb8f-a95ed32082a4")>]]


StaleElementReferenceException: Message: The element reference of <a id="Menu1_dgMenu_ctl03_LinkButton1" href="javascript:__doPostBack('Menu1$dgMenu$ctl03$LinkButton1','')"> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed


# Teste

Pegar subsessões

In [103]:
elements = driver.find_element_by_id("Menu1_dgMenu_ctl02_LinkButton1")
elements.click()
tabela = driver.find_element_by_id("dgDocumentos")
subsessoes = tabela.find_elements_by_tag_name("a")
for s in subsessoes:
    print(s.text)

CONTRATOS, EDITAIS E AVISOS
GOVERNADORIA
SECRETARIAS DE ESTADO


Pegar links html

In [106]:
elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
elements.click()
tabela = driver.find_element_by_id("dgDocumentos")
files = tabela.find_elements_by_tag_name("a")
links = [f.get_attribute("href") for f in files]
links

['http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190921&id_doc=659855',
 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190921&id_doc=659856']

Caminhar até o fim

In [114]:
elements = driver.find_element_by_id('Menu1_dgMenu')
sessoes = elements.find_elements_by_tag_name('a')

subsessoes_1 = navegar(sessoes[0])
subsessoes_2 = navegar(subsessoes_1[0])
subsessoes_3 = navegar(subsessoes_2[0])
subsessoes_4 = navegar(subsessoes_3[0])
subsessoes_5 = navegar(subsessoes_4[0])
subsessoes_5

[]